In [1]:
import torch
import numpy as np
import sys
sys.path.append('F:\\graduate_student\\T2_GANpropeller\\test5\\2_experiments\\FFD_PCA')
import draw
def guassian_kernel(source, target, kernel_mul=2.0, kernel_num=5, fix_sigma=None):
    '''
    将源域数据和目标域数据转化为核矩阵，即上文中的K
    Params: 
	    source: 源域数据（n * len(x))
	    target: 目标域数据（m * len(y))
	    kernel_mul: 
	    kernel_num: 取不同高斯核的数量
	    fix_sigma: 不同高斯核的sigma值
	Return:
		sum(kernel_val): 多个核矩阵之和
    '''
    n_samples = int(source.size()[0])+int(target.size()[0])# 求矩阵的行数，一般source和target的尺度是一样的，这样便于计算
    total = torch.cat([source, target], dim=0)#将source,target按列方向合并
    #将total复制（n+m）份
    total0 = total.unsqueeze(0).expand(int(total.size(0)), int(total.size(0)), int(total.size(1)))
    #将total的每一行都复制成（n+m）行，即每个数据都扩展成（n+m）份
    total1 = total.unsqueeze(1).expand(int(total.size(0)), int(total.size(0)), int(total.size(1)))
    #求任意两个数据之间的和，得到的矩阵中坐标（i,j）代表total中第i行数据和第j行数据之间的l2 distance(i==j时为0）
    L2_distance = ((total0-total1)**2).sum(2) 
    #调整高斯核函数的sigma值
    if fix_sigma:
        bandwidth = fix_sigma
    else:
        bandwidth = torch.sum(L2_distance.data) / (n_samples**2-n_samples)
    #以fix_sigma为中值，以kernel_mul为倍数取kernel_num个bandwidth值（比如fix_sigma为1时，得到[0.25,0.5,1,2,4]
    bandwidth /= kernel_mul ** (kernel_num // 2)
    bandwidth_list = [bandwidth * (kernel_mul**i) for i in range(kernel_num)]
    #高斯核函数的数学表达式
    kernel_val = [torch.exp(-L2_distance / bandwidth_temp) for bandwidth_temp in bandwidth_list]
    #得到最终的核矩阵
    return sum(kernel_val)#/len(kernel_val)
 
def mmd_rbf(source, target, kernel_mul=2.0, kernel_num=5, fix_sigma=None):
    '''
    计算源域数据和目标域数据的MMD距离
    Params: 
	    source: 源域数据（n * len(x))
	    target: 目标域数据（m * len(y))
	    kernel_mul: 
	    kernel_num: 取不同高斯核的数量
	    fix_sigma: 不同高斯核的sigma值
	Return:
		loss: MMD loss
    '''
    
    batch_size = source.size()[0]#一般默认为源域和目标域的batchsize相同
    kernels = guassian_kernel(source, target,
        kernel_mul=kernel_mul, kernel_num=kernel_num, fix_sigma=fix_sigma)
    #根据式（3）将核矩阵分成4部分
    XX = kernels[:batch_size, :batch_size]
    YY = kernels[batch_size:, batch_size:]
    XY = kernels[:batch_size, batch_size:]
    YX = kernels[batch_size:, :batch_size]
    loss = torch.mean(XX + YY - XY -YX)
    return loss.numpy() #因为一般都是n==m，所以L矩阵一般不加入计算

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
path = 'F:\\graduate_student\\T2_GANpropeller\\test5\\0_database\\mixed_model\\'

In [5]:
import random
import matplotlib.pyplot as plt
def easy_surface(foil_data,ax, path = ''):
    # p = foil_data
    foil_data = foil_data.reshape(-1,3)
    x = foil_data[:,0]
    y = foil_data[:,1]
    z = foil_data[:,2]


    # ax.set_xlim(0,1)
    # ax.set_ylim(-0.5,0.5)
    # ax.set_zlim(-0.5,0.5)

    # ax.plot_surface(x,y,z,rstride = 1,cstride = 1, cmap= 'gist_yarg',alpha = 0.3)
    ax.scatter3D(x,y,z,s=0.5,alpha = 0.1,c='b')
    return ax

def get_meshcloud(new_meshs,save_path,num  = 100):

    posi =random.sample(range(1,new_meshs.shape[0]),num)
    new_meshs = new_meshs[posi]
    fig = plt.figure(figsize=(10, 7))
    ax = plt.axes(projection="3d")
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.zaxis.set_visible(False)
    for i in new_meshs:
        ax = easy_surface(i,ax)

    ax.view_init(elev=80, azim=0)
    plt.axis('off')
    plt.savefig(save_path)
    plt.close()

In [17]:

column  = np.load(path + 'Column.npy')
tangent = np.load(path + 'Tangent.npy')
tapered = np.load(path + 'Tapered.npy')
column = column[:,:,10:,:]
tangent = tangent[:,:,10:,:]
tapered = tapered[:,:,10:,:]
a = column[700:900,:,:,:]
b = tangent[700:900,:,:,:]
c = tapered[700:900,:,:,:]

real = np.concatenate((column,tangent,tapered,a,a,b,b,c,c),0)


print(real.shape)
# real = np.concatenate((tangent0,tangent0,tangent0),0)
get_meshcloud(real,'test',200)

(4200, 18, 58, 3)


In [40]:

s1 = []
s2 = []
s3 = []
s4 = []

for i in range(10):
    a = random.sample(range(1,1000),100)
    d1 = column[a,:]
    d2 = tangent[a,:]
    d3 = tapered[a,:]

    mmd1 = mmd_rbf(d1,d2)
    mmd2 = mmd_rbf(d2,d3)
    mmd3 = mmd_rbf(d1,d3)

    s1.append(mmd1)
    s2.append(mmd2)
    s3.append(mmd3)

s1 = np.array(s1)
s2 = np.array(s2)
s3 = np.array(s3)

print(np.mean(s1),np.std(s1))
print(np.mean(s2),np.std(s2))
print(np.mean(s3),np.std(s3))

0.8528748138368991 0.14529380279604476
1.0458522159877126 0.1698616658681942
2.3574473258201714 0.2695328419169131


In [13]:
import random
real  =  torch.from_numpy(np.load('data_set/easy_real.npy'))

fake_ffd = torch.from_numpy(np.load('data_set/easy_fpgan_three.npy'))
upper_line = 2.2657
print(real.shape,fake_ffd.shape)
s_ffd = []
s_self = []
p = 200
for i in range(20):
   
    a1 = random.sample(range(1,real.shape[0]),p)
    a2 = random.sample(range(1,fake_ffd.shape[0]),p)
    # a3 = random.sample(range(1,tangent.shape[0]),p)
    
    d1 = real[a1,:].reshape(p,-1)
    d2 = fake_ffd[a2,:].reshape(p,-1)
    # d3 = tangent[a3,:].reshape(p,-1)
    
    s_ffd.append(upper_line/mmd_rbf(d1,d2))
    # s_self.append(mmd_rbf(d1,d3))

s_ffd = np.array(s_ffd)
# s_self = np.array(s_self)

# print(np.mean(s_self),np.std(s_self))

torch.Size([3000, 18, 68, 3]) torch.Size([3000, 18, 68, 3])
198.5687547065149 127.11890959304182


In [79]:
import random
real  =  torch.from_numpy(np.load('data_set/easy_real.npy'))
upper_line = 2.2657
def get_mmd(real,fake_ffd,p=200,num=20):
    s_ffd = []
    
    for i in range(num):
    
        a1 = random.sample(range(1,real.shape[0]),p)
        a2 = random.sample(range(1,fake_ffd.shape[0]),p)
        d1 = real[a1,:].reshape(p,-1)
        d2 = fake_ffd[a2,:].reshape(p,-1)
        # d3 = tangent[a3,:].reshape(p,-1)
        
        s_ffd.append(upper_line/mmd_rbf(d1,d2))
    s_ffd = np.array(s_ffd)
    return np.mean(s_ffd),np.std(s_ffd)

In [80]:
name_list1 = ['base1','base2','base3','base5','base7','base9']
name_path  = 'F:\\graduate_student\\T2_GANpropeller\\test5\\1_model\\result\\test_pic\\'
import random
real  =  torch.from_numpy(np.load('data_set/easy_real.npy'))
for i in name_list1:
    fake_ffd = torch.from_numpy(np.load(name_path + i + '.npy'))
    print(get_mmd(real,fake_ffd,100,3))

(2.022617576760678, 0.17593465440976275)
(3.902312234943809, 0.12575833161185965)
(5.130394036238759, 0.3421242479469372)
(6.303164808063722, 0.21814429422113618)
(10.933886336015641, 3.329764826944521)
(12.244826520971586, 2.90070382458126)


In [82]:
name_list1 = ['easy8','easy16','easy32','easy64']
name_path  = 'F:\\graduate_student\\T2_GANpropeller\\test5\\1_model\\result\\test_pic\\'
import random
real  =  torch.from_numpy(np.load('data_set/easy_real.npy'))
for i in name_list1:
    fake_ffd = torch.from_numpy(np.load(name_path + i + '.npy'))
    print(np.array([*get_mmd(real,fake_ffd,100,10)]))

[5.31233812 0.63313783]
[5.13806604 0.58118746]
[4.97511843 0.73060158]
[4.91094024 0.79473847]


In [49]:
name_list1 = ['base332','base371','base531','base571']
name_path  = 'F:\\graduate_student\\T2_GANpropeller\\test5\\2_experiments\\FFD_PCA\\ffd_vari_dim\\'

for i in name_list1:
    fake_ffd = torch.from_numpy(np.load(name_path + i + '.npy'))
    print(np.array([*get_mmd(real,fake_ffd,100,10)])/3)

[1.24732956 0.19859163]
[1.19400771 0.12631823]
[1.21930977 0.08748066]
[1.33749348 0.17015989]


In [50]:
name_list1 = ['base1','base2','base3','base5','base7','base9']
name_path  = 'F:\\graduate_student\\T2_GANpropeller\\test5\\2_experiments\\FFD_PCA\\ffd_vari_easy\\'

for i in name_list1:
    fake_ffd = torch.from_numpy(np.load(name_path + i + '.npy'))
    print(np.array([*get_mmd(real,fake_ffd,100,10)])/10)

[0.66594176 0.08781105]
[0.66852299 0.10217645]
[0.86778923 0.085371  ]
[1.03564413 0.09866828]
[0.94233266 0.11818045]
[0.92730218 0.07722494]


In [83]:
import matplotlib.pyplot as plt

y = [1.9797,3.6330,4.9859,6.0987,7.1561,7.4985]
x = [1,2,3,5,7,9]
std = [0.09433,0.2124,0.5231,0.6493,0.870,0.894]

y1 = [0.6659,0.6685,0.8678,1.036,0.9434,0.9273]
x1 = x
std1 = [0.0547,0.0474,0.0631,0.0744,0.0535,0.0693]

def draw_MMD(x,y,std,x1,y1,std1,x_str):
    with plt.style.context(['science','no-latex']):
        plt.figure(figsize=(20, 10))
        
        plt.errorbar(x,y,std,elinewidth=4)
        plt.errorbar(x1,y1,std1,elinewidth=4)
        plt.show()

# draw_MMD(x,y,std,x1,y1,std1,'')

# 测试直线度

In [108]:
def get_onestraight(points):
    Sum_X=0.0
    Sum_Y=0.0
    Sum_Z=0.0
    Sum_XZ=0.0
    Sum_YZ=0.0
    Sum_Z2=0.0
    
    xi = points[:,0]
    yi = points[:,1]
    zi = points[:,2]
    Sum_X = np.sum(xi)
    Sum_Y = np.sum(yi)
    Sum_Z = np.sum(zi)
    Sum_XZ = np.sum(xi*zi)
    Sum_YZ = np.sum(yi*zi)
    Sum_Z2 = np.sum(zi**2)
 
    n = points.shape[0] # 点数
    den = n*Sum_Z2 - Sum_Z * Sum_Z # 公式分母
    k1 = (n*Sum_XZ - Sum_X * Sum_Z)/ den
    b1 = (Sum_X - k1 * Sum_Z)/n
    k2 = (n*Sum_YZ - Sum_Y * Sum_Z)/ den
    b2 = (Sum_Y - k2 * Sum_Z)/n
    
    Q1 = np.sum((xi - k1*zi - b1)**2)
    Q2 = np.sum((yi - k2*zi - b2)**2)

    return Q1+Q2

import random
def get_allstraight(points,p = 100):
    a1 = random.sample(range(1,points.shape[0]),p)
    points = points[a1]
    
    straight = []
    for i in points:

        num_line = i.shape[0]

        for j in range(num_line):

            line = i[j]
            tmp = get_onestraight(line)
            straight.append(tmp)
    # print(straight)
    return np.mean(np.array(straight))

In [106]:

path = 'F:\\graduate_student\\T2_GANpropeller\\test5\\0_database\\mixed_model\\easy_real.npy'

real = np.load(path)
test = real[0,:,2,:]


a = get_onestraight(test)
b = get_allstraight(real)
print(a,b)

80.53980491425924 1.799880562511252e-16


In [109]:
name_list1 = ['base332','base371','base531','base571']
name_path  = 'F:\\graduate_student\\T2_GANpropeller\\test5\\2_experiments\\FFD_PCA\\ffd_vari_dim\\'

for i in name_list1:
    fake_ffd = np.load(name_path + i + '.npy').reshape(-1,18,68,3)
    print(get_allstraight(fake_ffd))

1765.0345030062135
2018.5048421750375
1794.7912573751616
1701.2695969541583


In [125]:
name_list1 = ['easy8','easy16','easy32','easy64']
name_path  = 'F:\\graduate_student\\T2_GANpropeller\\test5\\1_model\\result\\test_pic\\'
for i in name_list1:
    fake_ffd = np.load(name_path + i + '.npy').reshape(-1,18,68,3)
    print(get_allstraight(fake_ffd))

231.83107777382816
287.6544062469509
212.8753705192395
225.87780645152975
